# Deploy VARCO LLM SMALL IST Model from AWS Marketplace 




VARCO LLM 2.0 is NCSOFT's large language model that can be applied to the development of various natural language processing-based AI services 
such as text generation, question answering, chatbots, summarization, and information extraction.
NCSOFT's VARCO LLM 2.0 was developed with our own technology, including data construction, pre-training, instruction tuning and alignment tuning.
We evaluated VARCO LLM 2.0 on various NLP tasks and its performance has significantly improved compared to VARCO LLM 1.0, and it boasts the highest performance among other Korean LLMs of similar sizes. 
In particular, it has been trained to be used in high-level natural language processing applications such as creative writing, summarization, question and answering, chatbots and translation, and shows high performance in related quantitative indicators.
For inquiries regarding further performance improvement or collaboration for service applications, please contact us by email (varco_llm@ncsoft.com).

This sample notebook shows you how to deploy [varco llm](https://aws.amazon.com/marketplace/pp/prodview-nnewbvmwmt2jy)  using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
3. [Clean-up](#3.-Clean-up)

    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package [listing page](https://aws.amazon.com/marketplace/pp/prodview-nnewbvmwmt2jy)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn = "arn:aws:sagemaker:us-west-2:594846645681:model-package/varco-llm-small-ist-2-0-28a5becf8e673ff884afbaa3e562fcc4"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import numpy as np
import io

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "varco-llm-2-0-model"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.4xlarge"
)
batch_transform_inference_instance_type = (
    "ml.g4dn.12xlarge"
)

### A.Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

------------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B.Create input payload

In [6]:
input = {
    "repetition_penalty": 1.05,
    "temperature": 0.3,
    "top_k": 50,
    "top_p": 1,
    "text": "안녕"
}

### C. Perform real-time inference

##### C-1. Stream Inference Example

In [8]:
class VarcoInferenceStream():
    def __init__(self, sagemaker_runtime, endpoint_name):
        self.sagemaker_runtime = sagemaker_runtime
        self.endpoint_name = endpoint_name

    def stream_inference(self, request_body):
        # Gets a streaming inference response
        # from the specified model endpoint:
        response = self.sagemaker_runtime\
            .invoke_endpoint_with_response_stream(
                EndpointName=self.endpoint_name,
                Body=json.dumps(request_body),
                ContentType="application/json"
        )
        # Gets the EventStream object returned by the SDK:
        for body in response["Body"]:
            raw = body['PayloadPart']['Bytes']
            yield raw.decode()


sm_runtime = boto3.client("sagemaker-runtime")
varco_inference_stream = VarcoInferenceStream(sm_runtime, model_name)
stream = varco_inference_stream.stream_inference(input)
for part in stream:
    print(part, end='')

네 안녕하세요

## 3. Clean-up

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

### A. Delete the endpoint

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

### B. Delete the model

In [ ]:
model.delete_model()

### C. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

